In [14]:
#@title Imports
import cv2
from google.colab.patches import cv2_imshow
from google.colab import drive
import numpy as np
import matplotlib.pyplot as plt
import math


In [15]:
# 10 points from house 1 and house 2

drive.mount("/content/drive/")

coordinates_path = '/content/drive/MyDrive/CV-project2/coords.txt'

with open(coordinates_path) as f:
    world_coordinates = f.read()
    #print(world_coordinates)

lines = world_coordinates.split('\n')

real_world = []

for line in lines:
    coordinates = line.split()

    coordinates = [float(coord) for coord in coordinates]
    real_world.append(coordinates)

test_real_world = real_world[:-1]
# to remove the empty column
real_world = real_world[:-1]
real = np.array(real_world)
real_world = np.delete(real, -1, axis=1)
# print(real_world_matrix)


house1_10points = np.array([[286, 408 , 1],
                           [414, 265, 1],
                           [482, 272, 1],
                           [607, 60, 1],
                           [750, 213, 1],
                           [574, 483, 1],
                           [410, 56, 1],
                           [679, 336, 1],
                           [276, 243, 1],
                           [734, 392, 1]])

house2_10points = np.array([[286, 443, 1],
                           [342, 286, 1],
                           [375, 286, 1],
                           [689, 70, 1],
                           [807, 246, 1],
                           [387, 536, 1],
                           [341, 55, 1],
                           [661, 389, 1],
                           [282, 266, 1],
                           [790, 478, 1]])

# the five points for task
points_house1 = np.array([[232,409,1 ],
                          [375,344, 1],
                          [414, 431,1],
                          [638,347,1],
                          [573,244,1]])
points_house2 = np.array([[238, 440, 1],
                          [325,374,1],
                          [343,473,1],
                          [573, 395, 1],
                          [420, 253,1]])



Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [24]:
from os import F_TLOCK
#@title 8-point algorithm

def fundamental_matrix(x1, x2):

  #matrix A is going to be of shape 7 x 9
  A = np.zeros((x1.shape[0],9))

  #fill the matrix
  for i in range(x1.shape[0]):
    x, y = x1[i, :2]
    x_prim, y_prim = x2[i, :2]

    A[i] = np.array([[np.dot(x_prim, x), np.dot(x_prim, y), x_prim, np.dot(y_prim, x), np.dot(y_prim, y), y_prim, x, y, 1]])
    # A[i] = np.array([[np.dot(x_prim, x), np.dot(x_prim, y), x, np.dot(y, x_prim), np.dot(y, y_prim), y, x_prim, y_prim, 1]])

  # svd in order to minimise the error
  _, _, V = np.linalg.svd(A)


  #fundamental matrix in the eigenvector which corresponds to the smallest eigenvalue
  # or as said in the project the last column of vector V
  fund_matrix = np.reshape(V[-1],(3,3))
  # print(np.linalg.norm(A@fund_matrix))

  print(np.linalg.norm(fund_matrix))

  #constraint eforcment using svd
  # split the fundamental matrix in U, D, V matrices where D is the diagonal matrix
  U, D, V = np.linalg.svd(fund_matrix)
  # singolare restriction
  # set the smallest singular value to zero to encorce that the determinant is zero
  D[-1] = 0
  F_prime = U @ np.diag(D)@ V
  print(np.linalg.norm(fund_matrix - F_prime, 'fro'))
  print('-------')

  # fundamental matrix after denormalization
  # F = np.dot(T2.T, np.dot(F_prime, T1))
  return F_prime



fundamental_matrix = fundamental_matrix(house1_10points, house2_10points)
print(f'Fundamental Matrix :\n{fundamental_matrix}')
print('-------')
print(f'Fundamental Matrix Rank:\n{np.linalg.matrix_rank(fundamental_matrix)}')

1.0000000000000002
1.1976076871429756e-06
-------
Fundamental Matrix :
[[ 8.02338400e-07  1.62719225e-06 -2.82774603e-03]
 [-9.23174966e-06 -1.74627511e-06  2.33948515e-02]
 [ 5.09047041e-03 -2.11376345e-02 -9.99485854e-01]]
-------
Fundamental Matrix Rank:
2


In [17]:
# to test the foundamental matrix we do the x'Fx = 0

def check_fundamental_matrix(F, x1, x2):
    for i in range(x1.shape[0]):
        x1_hom = x1[i]  # already in homogenous form
        x2_hom = x2[i]

        error = np.dot(x2_hom.T, np.dot(F, x1_hom))
        print("error: ", error)
        if np.abs(error) > 1e-6:  # tolerance threshold
            return False

    return True

is_valid = check_fundamental_matrix(fundamental_matrix, house1_10points, house2_10points)
print(f'Is the fundamental matrix valid? {is_valid}')


error:  0.15764657050870667
Is the fundamental matrix valid? False


In [18]:
#@title 3D reconstruction

from scipy.linalg import null_space

# finding the world coordinates

# Left Nullspace: It's similar to the nullspace, but instead of multiplying the matrix by
# the vector on the right (A*v = 0), we multiply the matrix by the vector on the left (v^T*A = 0).

#  solves Av=0
e_prime = null_space(fundamental_matrix.T)
e_prime_unflatten = e_prime
# print(e_prime.T@fundamental_matrix)
# normalization step, to convert to homogeneous coordinates
e_prime = e_prime.flatten()
# print(e_prime)
e_prime /= e_prime[2]
# print(e_prime)

e_prime = np.array(e_prime)
e_prime_x = np.array([[0, -e_prime[2], e_prime[1]],
                     [e_prime[2], 0, -e_prime[0]],
                     [-e_prime[1], e_prime[0], 0]])



# [I | 0]
P = np.hstack([np.eye(3), np.zeros((3, 1))])
# print(P)

# [e'x F | e']
np.dot(e_prime_x, fundamental_matrix)
P_prime = np.hstack([np.dot(e_prime_x, fundamental_matrix), e_prime_unflatten])
# print(np.shape(P_prime))

# print("P: ", P, P.shape)


# x = PX
# x' = P'X
# x × PX = 0
# x' × P'X = 0

def triangulate_point(P, P_prime, x, x_prime):

    a = np.dot(np.cross(x, P[:, :3]), P)
    b = np.dot(np.cross(x_prime, P_prime[:, :3]), P_prime)

    # create matrix A for homogeneous equation system AX = 0
    A = np.vstack((a,b))

    _, _, V = np.linalg.svd(A)
    # convert from homogeneous coordinates to 3D Cartesian coordinates
    X = V[-1, :3] / V[-1, 3]

    return X

triangulated_points = []
for point1, point2 in zip(house1_10points, house2_10points):
    triangulated_point = triangulate_point(P, P_prime, point1, point2)
    triangulated_points.append(triangulated_point)


print(np.array(triangulated_points))



[[2.78097320e-03 3.96726297e-03 9.54451608e-06]
 [3.99162503e-03 2.55502850e-03 9.09879516e-06]
 [4.39326984e-03 2.47919206e-03 8.64676240e-06]
 [2.04617061e-03 2.02271097e-04 4.69510244e-06]
 [6.29754977e-03 1.78854420e-03 5.81812231e-06]
 [4.96138391e-03 4.17482368e-03 8.35533120e-06]
 [3.06519745e-03 4.18703633e-04 9.19795138e-06]
 [4.29054768e-03 2.12316028e-03 5.66897016e-06]
 [2.84724942e-03 2.50681842e-03 1.01344132e-05]
 [3.99728808e-03 2.13479320e-03 4.89595566e-06]]


In [19]:
#@title 3D homography

# Set up the system of linear equations
A = []

test_real_world = np.array(test_real_world)
triangulated_points = np.array(triangulated_points)
# print(len(triangulated_points))
# print(test_real_world)
# print(len(test_real_world))
# print()


for i in range(len(test_real_world)):
    x, y, z, _ = test_real_world[i]
    x_hat, y_hat, z_hat = triangulated_points[i]

    # Construct matrix A
    Ai = np.array([[x, y, z, 1, 0, 0, 0, 0, 0, 0, 0, 0, -x_hat*x, -x_hat*y, -x_hat*z, -x_hat],
                   [0, 0, 0, 0, x, y, z, 1, 0, 0, 0, 0, -y_hat*x, -y_hat*y, -y_hat*z, -y_hat],
                   [0, 0, 0, 0, 0, 0, 0, 0, x, y, z, 1, -z_hat*x, -z_hat*y, -z_hat*z, -z_hat]])

    A.append(Ai)

# Stack matrices A and vector b
A = np.vstack(A)

# Solve the system using SVD
u, s, vh = np.linalg.svd(A)
h = vh[-1, :]  # Take the last row of vh as the solution vector h
H = h.reshape(4, 4)

print('Estimated Homography:')
print(H)
# print(H.shape)
H_inverse_matrix = np.linalg.inv(H)
# print(H_inverse_matrix)



Estimated Homography:
[[-6.45069913e-04  3.64841572e-04 -1.22137520e-03  2.81263013e-03]
 [-1.12466814e-03  1.36870734e-04 -1.32804070e-03  2.63874362e-03]
 [-3.46068256e-06  4.70995791e-07 -3.09133629e-06  6.99774908e-06]
 [-3.35645642e-01  1.24506358e-01 -3.71082596e-01  8.56806813e-01]]


In [20]:
ones_column = np.ones((triangulated_points.shape[0], 1))
# Concatenate the ones_column to the matrix along the second axis
matrix_with_ones = np.hstack((triangulated_points, ones_column))

In [22]:
# World coordinates for five points images

correct_camera_p = np.dot(P, H_inverse_matrix)
correct_camera_p_prime = np.dot(P_prime, H_inverse_matrix)

print(f'correct camera p{correct_camera_p}')
print('------')
print(f'correct camera p prime{correct_camera_p_prime}')


Y_j = []
for point1, point2 in zip(points_house1, points_house2):
    triangulated_point = triangulate_point(correct_camera_p, correct_camera_p_prime, point1, point2)
    Y_j.append(triangulated_point)

Y_j = np.array(Y_j)
print('------')
print(Y_j)

# test = np.dot(correct_camera_p, matrix_with_ones.T)
# # print(np.shape(test))
# # print('-------')
# # print((test/test[2:]).T)


[[ 1.93420293e+03  2.01640778e+01 -1.64566710e+05 -5.06743022e+00]
 [-2.86740433e+03  6.73689330e+02 -1.90575867e+06  2.29028003e+01]
 [ 2.86761098e+03 -6.30635326e+03  2.61814243e+06 -1.13745401e+01]]
------
[[-5.17878351e+06  1.16524317e+07 -4.77521860e+09  2.01153826e+04]
 [ 4.18771615e+07 -9.42037041e+07  3.86066418e+10 -1.62656284e+05]
 [ 1.01975147e+06 -2.24303587e+06  9.31286921e+08 -4.04560186e+03]]
------
[[3.28381978e-03 3.07817226e-03 8.16222850e-06]
 [3.28440601e-03 3.07843204e-03 8.16221136e-06]
 [3.28446075e-03 3.07824112e-03 8.16169134e-06]
 [3.28491939e-03 3.07858049e-03 8.16200539e-06]
 [3.28545412e-03 3.07850277e-03 8.16123344e-06]]
